# <Feedback control of a cassie bipedal robot: walking, standing and riding a Segway> 

## 状态说明
$q_1$: hip roll, $q_2$: hip yaw, $q_3$: hip pitch, $q_4$: knee pitch, $q_5$: shin pitch, $q_6$: tarsus pitch, $q_7$: toe pitch

全维状态量 $q = [q_x, q_y, q_z, q_{yaw}, q_{pitch}, q_{roll}, q_{1L}, q_{2L},q_{3L}, q_{4L}, q_{5L}, q_{6L}, q_{7L}, q_{1R}, q_{2R},q_{3R}, q_{4R}, q_{5R}, q_{6R}, q_{7R}]$

$q_{yaw}, q_{pitch}, q_{roll}$ 根据cassie的IMU进行EKF滤波得到， $q_{1L}, ... q_{7L},q_{1R, ... q_{7R}}$根据电机编码器得到

$q_5, q_6$是被动关节（相当硬的弹簧连接四连杆），弹簧未压缩时：$q_5 = 0, q_6 = -q_4+13^\circ$


## 动力学模型
Lagrange 动力学方程为 

$$ D(q) \ddot{q} + H(q, \dot{q}) = Bu + J_{sp}^T(q) \tau_{sp} + J(q)^T \lambda $$


D(q)为广义质量矩阵， B为控制力矩矩阵，u是控制量，$H(q,\dot(q))$为速度和重力相关项， $J_{sp}(q) = \frac{\partial}{\partial q} \begin{bmatrix} q_{5L} \\ q_{4L}+q_{5L}+q_{6L} \\ q_{5R} \\ q_{4R}+q_{5R}+q_{6R} \end{bmatrix}$，$J_{sp}, \tau_{sp}$分别是弹簧Jacobian和弹簧力矩。$\lambda$是摩擦力（contact wrench）。

## Hybrid model for walking

walking包括单足支撑和双足支撑，双足支撑的时间很短

### 右脚单脚站立模型
这里将足端触地视为完整约束，为$ c_R{q}^T = [p_R^x, p_R^y, p_R^z, \psi_R^{yaw}, \psi_R^{pitch}] $，前三维是右脚在地面位置发生相对滑动，后两项是右脚在地面时pitch和yaw角不发生变化（cassie脚很窄，所以不限制roll角）,此时右脚接触Jacobian为 

$$ J_R(q) = \frac{\partial c_R(q)}{ \partial q} $$

接触动力学为：$J_R(q) \ddot{q} + \dot{J_R}(q, \dot{q}) \dot{q} = 0$，等式右边为0意味着将$c_R(q)$中的状态加速度限制为0。由于接触动力学方程满秩（rank = 5），因此可以将原动力学方程（20维）降秩为15DoF
### 双脚站立模型
摆动相对地高度为0，one-foot-stance变为two-foot-stance。双脚站立模型中左右脚均有完整的接触约束(10维等式约束)。

Note1：摆动相的脚需要大致与地面平行（$q_7$施加力矩），但是该力矩足够小，所以当机器人落地时，脚会被机器人自身重力克服$q_7$力矩压成贴合地面的角度。

Note2：摆动相的足端是否与地面发生碰撞检测依据：$q_5， q_6$的弹簧是否被压缩。碰撞仿真是通过离散映射的方式实现的，碰撞前后位置连续但是速度不连续。碰撞后，支撑相和摆动相的约束互换。

### 单脚到双脚的动力学切换模型
以$q^-, q^+$分别代表摆动相触地前后的状态，则触地前后的动力学关系满足如下约束，
$$ \begin{bmatrix}  D(q) & -J_L^T(q)\\ J_L(q) & 0 \end{bmatrix} \begin{bmatrix} \dot{q^+} \\ \delta F_L \end{bmatrix} = \begin{bmatrix} D(q)\dot{q^-} \\ 0 \end{bmatrix} $$
其中$\delta F_L$是碰撞冲击向量。因为D(q)满秩并且$J_L(q)$满秩，所以上式左边是可逆的。所以能够根据$q_-$求解$q_+$。


## walking on various terrains
### virtual constraints
virtual constraints的概念在于让被控物理量$h_0(q)$跟随给定物理量$h_d(\tau, \alpha)$，二者的差异为0就形成所谓的virtual constraints，即$$y = h(q,\tau,\alpha) = h_0(q)-h_d(\tau,\alpha)$$ 控制器的设计就是让所谓的virtual constraints渐进趋于0.
### 被控量选择
被控物理量：躯干朝向、支撑相和摆动相的长度、摆动腿朝向、摆动相foot朝向（这些物理量不依赖人形、鸵鸟形双足，是通用的双足步态表示）。根据Cassie的拓扑结构，选择$$h_0(q)=\left[\begin{array}{l} q_{\text {roll }} \\ q_{2 \mathrm{st}} \\ q_{\mathrm{pitch}} \\ q_{\mathrm{LL} \text { st }} \\ q_{\mathrm{LR} \text { sw }} \\ q_{2 \mathrm{sw}} \\ q_{\mathrm{LP} \text { sw }} \\ q_{\mathrm{LL} \mathrm{sw}} \\ q_{\mathrm{FP}} \mathrm{sw} \end{array}\right] \quad\left(\begin{array}{c} \text { torso roll } \\ \text { stance hip yaw } \\ \text { torso pitch } \\ \text { stance leg length } \\ \text { swing leg roll } \\ \text { swing hip yaw } \\ \text { swing leg pitch } \\ \text { swing leg length } \\ \text { swing foot pitch } \end{array}\right)$$
当前支撑相为右腿、摆动相为左腿时，根据论文Figure 3.b的leg length等定义以及机器人的物理配置参数，有正向运动学计算关系如下，
$$ \begin{align} & q_{\mathrm{LL} \text { st }} = \sqrt{0.5292cos(q_{4R}+0.035)+0.5301} \\
& q_{\mathrm{LR} \text{ sw}} = q_{roll}+q_{1L} \\
& q_{\mathrm{LP} \text { sw}} = -q_{pitch} + q_{3L} - arccos(\frac{0.5(cos(q_{4L}+0.035)+0.5292)}{\sqrt{0.5292cos(q_{4L}+0.035)+0.5301}})+0.1 \\
& q_{\mathrm{FP} \text{ sw}} = q_{pitch}+q_{7L} + 1.1 \end{align}$$
从上面的公式，可以根据被控量期望反解实际的驱动器$ q_i, i=1,2...7$。
### 不同速度下的步态集合以及步态控制（$h_d$是怎么定义的）
被控量$h_d$是根据步态库中不同步速下的步态离散插值得到的。virtual constraint对应一个步态（不同速度下有不同的步态设计），virtual constraint拥有设定的轨迹$h_{\tau, \alpha_i}^i$，其中$\alpha_i$是5阶贝塞尔多项式的系数矩阵。步态控制的一个周期包括左脚站立+右脚站立两个问题，总体上可以视为如下优化问题， $$ cost = \int_{\tau = 0}^{\tau = 1} (\parallel u\parallel^2+c|q_{pitch}|^2 + c|q_{roll}|^2+c|q_{1L}|^2+c|q_{2L}|^2+c|q_{1R}|^2+c|q_{2R}|^2) d\tau $$ 
上述cost function中，躯干的roll和pitch以及hip的roll和yaw都乘以系数c（c=10000），以此来保证优化后的步态能够使得roll和yaw的移动更小。步态优化求解过程中需要施加力矩约束、关节角度约束、关节角速度约束等，其他约束参考论文中table 1. 该约束问题可以使用MATLAB IPOPT工具进行求解。根据上述关系可以得到步态下的物理被控量期望，然后根据物理被控量期望可以映射到$q_i$（躯干控制量不用映射，直接控制对应电机）。virutal constraints的期望即通过上述过程获得。virtual constraints的收敛是通过PD控制器实现的，即对 $y = h_0(q)-h_d(\tau,\alpha)$的期望跟随过程施加PD控制作用$u = -K_D \tilde{y} -K_D \dot{\tilde{y}}$。对不同的步态，PD增益是相同的。

$q_{roll},q_{pitch}$二者定义在全局坐标系下，是通过髋关节的电机进行直接控制的。
### 步态库以及利用leg角度调整稳定双足
文章对Cassie机器人针对-0.5～1.0 m/s的速度区间，以0.25的间隔取了7个步态。Gait Library是根据这7个步态插值得到的一个相对较连续的步态库，插值参数是机器人的滤波矢状速度。

该部分内容原理参考<from 2D design of underactuated bipedal gaits to 3D implementation: walking with speed tracking>
humanoid对矢状面的速度和额状面的速度进行解耦的反馈控制，矢状面速度的调节是step by step地逼近速度期望（像一个离散积分器），矢状面的速度控制表现像一个线性化的倒立摆模型。对于给定的矢状面速度$v_{sag}$，则步态库的插值遵循下述贝塞尔线性插值关系，$$\begin{align} &\zeta(v_{sag}) = \frac{v_{sag}-v_{sag,i}}{v_{sag,i+1}-v_{sag,i}}, 1 \le i \le 7  \\  & \alpha(v_{sag}) = (1-\zeta(v_{sag}))\alpha_i + \zeta(v_{sag})\alpha_{i+1} \end{align}$$
所以步态库定义为 $$A = \left \{ \alpha(v_{sag}) \mid -0.5 \le v_{sag} \le 1) \right \}$$
步态控制具有近似中性稳定（不懂啥意思，论文也没细讲），仅仅通过调节摆动相的foot放置即可调节速度，即调节步幅。每一步调节步幅可以通过调节足端的前偏角增量实现（髋关节与足端的连线与重力方向的夹角），增量定义如下，
 $$\delta_{LA, tgt}^{sw} = K_p(v_{sag}[k]-v_{sag}^{ref}) + K_d (v_{sag}[k]-v_{sag}[k-1])$$
### 站立
双足机器人的重心为$p^{CoM} = (p_x^{CoM}, p_y^{CoM}, p_z^{CoM})$。其中$p_x^{CoM}$是通过调节foot的pitch实现的（$q_7$）。$p_y^{CoM}$是通过机器人的roll angle（roll angle是通过调节两条腿的伸展长度实现的，比如可以实现一条腿站在平地，一条腿站在台阶上）进行调节，控制期望一般维持0（a zero roll angle is equivalent to a centered $p_y^{CoM}$  when other joints are symmetric）；还有一个理由控制roll angle而不是直接控制$p_y^{CoM}$，是因为roll angle的信噪比要高于$p_y^{CoM}$。$p_z^{CoM}$是通过控制两条腿的平均站立高度实现的。hip roll和hip yaw被调节至常数值









